<a href="https://colab.research.google.com/github/ArayaTripathi/Gingivitis-detection-model/blob/main/Gingivitis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install roboflow
!pip install torch torchvision scikit-learn matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 72.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [23]:
from roboflow import Roboflow

rf = Roboflow(api_key="uwwTsnHAguK00gGGxInO")
project = rf.workspace("betic-project").project("new-gingivitis-classification-ojpsc")
version = project.version(3)
dataset = version.download("folder")  # downloads to /content/folder


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to New-Gingivitis-classification-3 in folder:: 100%|██████████| 303/303 [00:00<00:00, 6871.82it/s]


In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Set paths
data_dir = os.path.join("folder")  # or dataset.location
train_dir = os.path.join(data_dir, "/content/New-Gingivitis-classification-3/train")
valid_dir = os.path.join(data_dir, "/content/New-Gingivitis-classification-3/valid")

# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

class_names = train_dataset.classes


In [ ]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load ResNet-34
model = models.resnet34(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(class_names))  # Adjust output layer
model = model.to(device)


The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 188MB/s]


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50  # increase for better performance

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


Epoch 1/50, Loss: 1.9631608426570892
Epoch 2/50, Loss: 0.9597252309322357
Epoch 3/50, Loss: 0.8170333877205849
Epoch 4/50, Loss: 0.5646812561899424
Epoch 5/50, Loss: 0.437220711261034
Epoch 6/50, Loss: 0.4972009211778641
Epoch 7/50, Loss: 0.26440953090786934
Epoch 8/50, Loss: 0.5121491495519876
Epoch 9/50, Loss: 0.28941552294418216
Epoch 10/50, Loss: 0.35561258159577847
Epoch 11/50, Loss: 0.5579711254686117
Epoch 12/50, Loss: 0.6165427453815937
Epoch 13/50, Loss: 0.6390258967876434
Epoch 14/50, Loss: 0.5615229830145836
Epoch 15/50, Loss: 0.5709259286522865
Epoch 16/50, Loss: 0.7194061912596226
Epoch 17/50, Loss: 0.5540519077330828
Epoch 18/50, Loss: 0.48469293117523193
Epoch 19/50, Loss: 0.2686875695362687
Epoch 20/50, Loss: 0.18527246871963143
Epoch 21/50, Loss: 0.1087113672401756
Epoch 22/50, Loss: 0.11210769368335605
Epoch 23/50, Loss: 0.07401996990665793
Epoch 24/50, Loss: 0.0660010795108974
Epoch 25/50, Loss: 0.06326125195482746
Epoch 26/50, Loss: 0.07618417125195265
Epoch 27/50, 

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in valid_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(preds.cpu().numpy())

# Accuracy
accuracy = accuracy_score(y_true, y_pred)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# Sensitivity (Recall) & Specificity (for binary or per class)
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

print(f"\nAccuracy: {accuracy:.4f}")
for cls in class_names:
    print(f"{cls} - Sensitivity (Recall): {report[cls]['recall']:.4f}, Specificity: TBD")



Accuracy: 0.3929
Mild - Sensitivity (Recall): 0.7500, Specificity: TBD
Normal - Sensitivity (Recall): 0.4000, Specificity: TBD
moderate - Sensitivity (Recall): 0.0000, Specificity: TBD
severe - Sensitivity (Recall): 0.0000, Specificity: TBD


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [ ]:
# For binary classification:
tn, fp, fn, tp = cm.ravel()

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")


ValueError: too many values to unpack (expected 4)

In [ ]:
torch.save(model.state_dict(), "resnet34_gingivitis.pth")


# Task
Improve the accuracy of the image classification model to 89.3% or higher.

## Increase epochs

### Subtask:
Train the model for more epochs to allow it to learn more from the data.


**Reasoning**:
Increase the number of training epochs and rerun the training loop to improve model performance.



In [ ]:
num_epochs = 100  # increased for better performance

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

Epoch 1/100, Loss: 0.573186720488593
Epoch 2/100, Loss: 0.24257121235132217
Epoch 3/100, Loss: 0.45592578314244747
Epoch 4/100, Loss: 0.33081923332065344
Epoch 5/100, Loss: 0.15837876894511282
Epoch 6/100, Loss: 0.1655716998502612
Epoch 7/100, Loss: 0.10169835295528173
Epoch 8/100, Loss: 0.05799546022899449
Epoch 9/100, Loss: 0.04922559589613229
Epoch 10/100, Loss: 0.09207282646093518
Epoch 11/100, Loss: 0.07150943111628294
Epoch 12/100, Loss: 0.30699684913270175
Epoch 13/100, Loss: 0.18628934118896723
Epoch 14/100, Loss: 0.3311542687006295
Epoch 15/100, Loss: 0.1891329495701939
Epoch 16/100, Loss: 0.4863142156973481
Epoch 17/100, Loss: 0.35676152631640434
Epoch 18/100, Loss: 0.1824147398583591
Epoch 19/100, Loss: 0.10631818417459726
Epoch 20/100, Loss: 0.05088087031617761
Epoch 21/100, Loss: 0.041364516073372215
Epoch 22/100, Loss: 0.03305039164843038
Epoch 23/100, Loss: 0.2510677566751838
Epoch 24/100, Loss: 0.32998351426795125
Epoch 25/100, Loss: 0.13220992730930448
Epoch 26/100, Lo

**Reasoning**:
The model has been trained for more epochs. Now, evaluate the model's performance on the validation set again to see if the accuracy has improved and calculate the metrics as before.



In [ ]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in valid_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(preds.cpu().numpy())

# Accuracy
accuracy = accuracy_score(y_true, y_pred)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# Sensitivity (Recall) & Specificity (per class)
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

print(f"\nAccuracy: {accuracy:.4f}")
for cls in class_names:
    print(f"{cls} - Sensitivity (Recall): {report[cls]['recall']:.4f}, Specificity: TBD")



Accuracy: 0.6786
Mild - Sensitivity (Recall): 0.2500, Specificity: TBD
Normal - Sensitivity (Recall): 0.8500, Specificity: TBD
moderate - Sensitivity (Recall): 0.3333, Specificity: TBD
severe - Sensitivity (Recall): 0.0000, Specificity: TBD


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


**Reasoning**:
Calculate and print the specificity for each class using the confusion matrix and the classification report.



In [ ]:
for i, cls in enumerate(class_names):
    # Calculate Specificity for each class
    # Specificity for class i = TN / (TN + FP)
    # TP: True Positives (diagonal element cm[i, i])
    # FN: False Negatives (sum of row i excluding diagonal)
    # FP: False Positives (sum of column i excluding diagonal)
    # TN: True Negatives (sum of all elements excluding row i and column i)

    TP = cm[i, i]
    FN = np.sum(cm[i, :]) - TP
    FP = np.sum(cm[:, i]) - TP
    TN = np.sum(cm) - TP - FN - FP

    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0.0

    print(f"{cls} - Sensitivity (Recall): {report[cls]['recall']:.4f}, Specificity: {specificity:.4f}")


Mild - Sensitivity (Recall): 0.2500, Specificity: 0.8333
Normal - Sensitivity (Recall): 0.8500, Specificity: 0.5000
moderate - Sensitivity (Recall): 0.3333, Specificity: 0.9600
severe - Sensitivity (Recall): 0.0000, Specificity: 1.0000


## Implement data augmentation

### Subtask:
Apply data augmentation techniques to increase the size and variability of the training dataset, which can help the model generalize better.


**Reasoning**:
Apply data augmentation to the training data by defining new transforms and updating the training dataset and dataloader.



In [ ]:
from torchvision import transforms

# Transforms with augmentation for training data
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
])

# Transforms for validation data (no augmentation)
valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Update Datasets with new transforms
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
valid_dataset = datasets.ImageFolder(valid_dir, transform=valid_transform)

# Re-create Dataloader for training data
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

## Fine-tune the learning rate

### Subtask:
Experiment with different learning rates to find one that allows the model to converge more effectively.


**Reasoning**:
Define a list of learning rates and iterate through them, training and evaluating the model for each to find the best one.



In [ ]:
learning_rates = [0.01, 0.001, 0.0001]
best_accuracy = 0
best_lr = None

for lr in learning_rates:
    print(f"Training with learning rate: {lr}")

    # Redefine optimizer with current learning rate
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Train the model for a fixed number of epochs
    num_epochs_experiment = 20  # Fixed number of epochs for the experiment
    for epoch in range(num_epochs_experiment):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"  Epoch {epoch+1}/{num_epochs_experiment}, Loss: {running_loss/len(train_loader)}")

    # Evaluate the model
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.numpy())
            y_pred.extend(preds.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy with learning rate {lr}: {accuracy:.4f}\n")

    # Keep track of the best learning rate
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_lr = lr

print(f"Best learning rate found: {best_lr} with accuracy: {best_accuracy:.4f}")


Training with learning rate: 0.01
  Epoch 1/20, Loss: 2.423934794962406
  Epoch 2/20, Loss: 1.462270051240921
  Epoch 3/20, Loss: 1.3457897156476974
  Epoch 4/20, Loss: 1.288005106151104
  Epoch 5/20, Loss: 1.4155889824032784
  Epoch 6/20, Loss: 1.2819211706519127
  Epoch 7/20, Loss: 1.1876063644886017
  Epoch 8/20, Loss: 1.4824185445904732
  Epoch 9/20, Loss: 1.2802995890378952
  Epoch 10/20, Loss: 1.1602602005004883
  Epoch 11/20, Loss: 1.2141933739185333
  Epoch 12/20, Loss: 1.1198775395751
  Epoch 13/20, Loss: 1.136034019291401
  Epoch 14/20, Loss: 1.0897700265049934
  Epoch 15/20, Loss: 1.282431498169899
  Epoch 16/20, Loss: 1.3455937877297401
  Epoch 17/20, Loss: 1.1318335011601448
  Epoch 18/20, Loss: 1.116824246942997
  Epoch 19/20, Loss: 1.1263310983777046
  Epoch 20/20, Loss: 1.1300238221883774
Accuracy with learning rate 0.01: 0.7143

Training with learning rate: 0.001
  Epoch 1/20, Loss: 1.0072174817323685
  Epoch 2/20, Loss: 1.1385728791356087
  Epoch 3/20, Loss: 1.0875823

## Use a different optimizer

### Subtask:
Try using a different optimizer, such as SGD with momentum or RMSprop, which might be better suited for this task.


**Reasoning**:
Define and iterate through different optimizers, training and evaluating the model with each to find the best one based on accuracy.



In [ ]:
optimizers_to_experiment = {
    "SGD_momentum": optim.SGD(model.parameters(), lr=best_lr, momentum=0.9),
    "RMSprop": optim.RMSprop(model.parameters(), lr=best_lr),
    "Adam": optim.Adam(model.parameters(), lr=best_lr) # Include Adam for comparison
}

best_optimizer_name = None
best_optimizer_accuracy = best_accuracy # Initialize with the best accuracy from the previous step

for optimizer_name, optimizer in optimizers_to_experiment.items():
    print(f"Training with optimizer: {optimizer_name}")

    # Train the model for a fixed number of epochs
    for epoch in range(num_epochs_experiment):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"  Epoch {epoch+1}/{num_epochs_experiment}, Loss: {running_loss/len(train_loader)}")

    # Evaluate the model
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.numpy())
            y_pred.extend(preds.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy with {optimizer_name}: {accuracy:.4f}\n")

    # Keep track of the best optimizer
    if accuracy > best_optimizer_accuracy:
        best_optimizer_accuracy = accuracy
        best_optimizer_name = optimizer_name

print(f"Best optimizer found: {best_optimizer_name} with accuracy: {best_optimizer_accuracy:.4f}")

Training with optimizer: SGD_momentum
  Epoch 1/20, Loss: 0.9382601007819176
  Epoch 2/20, Loss: 0.8806280195713043
  Epoch 3/20, Loss: 0.8828477337956429
  Epoch 4/20, Loss: 0.8004813268780708
  Epoch 5/20, Loss: 0.7817757576704025
  Epoch 6/20, Loss: 0.9125477746129036
  Epoch 7/20, Loss: 0.9550539329648018
  Epoch 8/20, Loss: 0.8654084205627441
  Epoch 9/20, Loss: 0.8151852414011955
  Epoch 10/20, Loss: 0.8707642108201981
  Epoch 11/20, Loss: 0.8234695717692375
  Epoch 12/20, Loss: 0.8162484541535378
  Epoch 13/20, Loss: 0.8198707103729248
  Epoch 14/20, Loss: 0.8612731471657753
  Epoch 15/20, Loss: 0.8850715458393097
  Epoch 16/20, Loss: 0.8292203322052956
  Epoch 17/20, Loss: 0.9766038581728935
  Epoch 18/20, Loss: 0.847256176173687
  Epoch 19/20, Loss: 0.844158947467804
  Epoch 20/20, Loss: 0.8270358219742775
Accuracy with SGD_momentum: 0.6786

Training with optimizer: RMSprop
  Epoch 1/20, Loss: 0.9763980358839035
  Epoch 2/20, Loss: 1.08404640853405
  Epoch 3/20, Loss: 0.974472

## Adjust model architecture

### Subtask:
Adjust the model architecture by using a more powerful pre-trained model to improve classification accuracy.


**Reasoning**:
Load the ResNet-50 model with pre-trained weights, freeze its parameters, replace the final layer, and move it to the appropriate device as per the instructions.



In [ ]:
import torch.nn as nn
from torchvision import models

# Load ResNet-50
resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

# Freeze parameters
for param in resnet50.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, len(class_names))

# Move model to device
model = resnet50.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 108MB/s]


## Fine-tune the learning rate

### Subtask:
Experiment with different learning rates to find one that allows the model to converge more effectively with the new ResNet-50 architecture.


**Reasoning**:
Define a list of learning rates, initialize variables for tracking the best accuracy and learning rate, and then iterate through the learning rates to train and evaluate the model, updating the best values found.



In [ ]:
learning_rates = [0.01, 0.001, 0.0001]
best_accuracy = 0
best_lr = None
num_epochs_experiment = 20

for lr in learning_rates:
    print(f"Training with learning rate: {lr}")

    # Redefine optimizer with current learning rate (using Adam as it performed well)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Train the model for a fixed number of epochs
    for epoch in range(num_epochs_experiment):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"  Epoch {epoch+1}/{num_epochs_experiment}, Loss: {running_loss/len(train_loader)}")

    # Evaluate the model
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.numpy())
            y_pred.extend(preds.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy with learning rate {lr}: {accuracy:.4f}\n")

    # Keep track of the best learning rate
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_lr = lr

print(f"Best learning rate found: {best_lr} with accuracy: {best_accuracy:.4f}")


Training with learning rate: 0.01
  Epoch 1/20, Loss: 4.788638070225716
  Epoch 2/20, Loss: 3.194862797856331
  Epoch 3/20, Loss: 1.8934206068515778
  Epoch 4/20, Loss: 1.4486229121685028
  Epoch 5/20, Loss: 1.285699762403965
  Epoch 6/20, Loss: 0.942675031721592
  Epoch 7/20, Loss: 1.1941833719611168
  Epoch 8/20, Loss: 1.3829333782196045
  Epoch 9/20, Loss: 0.9839236587285995
  Epoch 10/20, Loss: 1.1318470537662506
  Epoch 11/20, Loss: 1.0379326865077019
  Epoch 12/20, Loss: 0.8577755987644196
  Epoch 13/20, Loss: 1.035595752298832
  Epoch 14/20, Loss: 1.1653383672237396
  Epoch 15/20, Loss: 1.009760294109583
  Epoch 16/20, Loss: 0.9517651051282883
  Epoch 17/20, Loss: 0.6601657047867775
  Epoch 18/20, Loss: 1.0800646506249905
  Epoch 19/20, Loss: 1.2210949137806892
  Epoch 20/20, Loss: 1.0486162453889847
Accuracy with learning rate 0.01: 0.5000

Training with learning rate: 0.001
  Epoch 1/20, Loss: 0.7541013769805431
  Epoch 2/20, Loss: 0.6002805158495903
  Epoch 3/20, Loss: 0.4598

## Use a different optimizer

### Subtask:
Try using a different optimizer, such as SGD with momentum or RMSprop, which might be better suited for this task with the ResNet-50 architecture.


**Reasoning**:
Define the optimizers to experiment with, initialize variables to track the best optimizer, and then iterate through the optimizers, train the model for a fixed number of epochs, evaluate the accuracy, and update the best optimizer if the current one performs better.



In [ ]:
optimizers_to_experiment = {
    "SGD_momentum": optim.SGD(model.parameters(), lr=best_lr, momentum=0.9),
    "RMSprop": optim.RMSprop(model.parameters(), lr=best_lr),
    "Adam": optim.Adam(model.parameters(), lr=best_lr) # Include Adam for comparison
}

best_optimizer_name = None
best_optimizer_accuracy = best_accuracy # Initialize with the best accuracy from the previous step
num_epochs_experiment = 20 # Use the same number of epochs as in the learning rate tuning

for optimizer_name, optimizer in optimizers_to_experiment.items():
    print(f"Training with optimizer: {optimizer_name}")

    # Train the model for a fixed number of epochs
    for epoch in range(num_epochs_experiment):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"  Epoch {epoch+1}/{num_epochs_experiment}, Loss: {running_loss/len(train_loader)}")

    # Evaluate the model
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.numpy())
            y_pred.extend(preds.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy with {optimizer_name}: {accuracy:.4f}\n")

    # Keep track of the best optimizer
    if accuracy > best_optimizer_accuracy:
        best_optimizer_accuracy = accuracy
        best_optimizer_name = optimizer_name

print(f"Best optimizer found: {best_optimizer_name} with accuracy: {best_optimizer_accuracy:.4f}")

Training with optimizer: SGD_momentum
  Epoch 1/20, Loss: 0.5303780511021614
  Epoch 2/20, Loss: 0.44337386824190617
  Epoch 3/20, Loss: 0.3674008287489414
  Epoch 4/20, Loss: 0.3870375193655491
  Epoch 5/20, Loss: 0.3291462976485491
  Epoch 6/20, Loss: 0.41386925615370274
  Epoch 7/20, Loss: 0.44887036085128784
  Epoch 8/20, Loss: 0.46327805891633034
  Epoch 9/20, Loss: 0.4251550640910864
  Epoch 10/20, Loss: 0.36350465193390846
  Epoch 11/20, Loss: 0.41404012218117714
  Epoch 12/20, Loss: 0.34882502630352974
  Epoch 13/20, Loss: 0.33676775358617306
  Epoch 14/20, Loss: 0.3414156176149845
  Epoch 15/20, Loss: 0.3836707826703787
  Epoch 16/20, Loss: 0.36359394900500774
  Epoch 17/20, Loss: 0.5164346769452095
  Epoch 18/20, Loss: 0.42734207957983017
  Epoch 19/20, Loss: 0.43782669492065907
  Epoch 20/20, Loss: 0.36716214194893837
Accuracy with SGD_momentum: 0.8214

Training with optimizer: RMSprop
  Epoch 1/20, Loss: 1.858758706599474
  Epoch 2/20, Loss: 0.7283057197928429
  Epoch 3/20,

## Evaluate with more metrics

### Subtask:
Calculate and analyze additional evaluation metrics like precision, F1-score, and per-class accuracy to get a more comprehensive understanding of the model's performance.


**Reasoning**:
Calculate and print the classification report to get a detailed breakdown of precision, recall, F1-score, and support for each class, which is necessary to understand the model's performance beyond overall accuracy.



In [ ]:
from sklearn.metrics import classification_report

# Calculate and print the classification report
report = classification_report(y_true, y_pred, target_names=class_names)
print("\nClassification Report:")
print(report)

# Analyze the report (this will be done manually based on the printed output)


Classification Report:
              precision    recall  f1-score   support

        Mild       0.50      0.50      0.50         4
      Normal       0.91      1.00      0.95        20
    moderate       1.00      0.67      0.80         3
      severe       0.00      0.00      0.00         1

    accuracy                           0.86        28
   macro avg       0.60      0.54      0.56        28
weighted avg       0.83      0.86      0.84        28



Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


## Increase epochs

### Subtask:
Train the model for more epochs to allow it to learn more from the data.


**Reasoning**:
Train the model for more epochs using the best optimizer and learning rate found in previous steps to see if it improves accuracy.



In [ ]:
# Use the best optimizer and learning rate from previous experiments
optimizer = optimizers_to_experiment["Adam"] # Based on previous experiment results
optimizer.lr = best_lr # Use the best learning rate found

num_epochs = 100 # Increased for more training

print(f"Training with Adam optimizer and learning rate: {best_lr} for {num_epochs} epochs.")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


Training with Adam optimizer and learning rate: 0.001 for 100 epochs.
Epoch 1/100, Loss: 0.5038147885352373
Epoch 2/100, Loss: 0.4334309007972479
Epoch 3/100, Loss: 0.31455086916685104
Epoch 4/100, Loss: 0.33859183453023434
Epoch 5/100, Loss: 0.3904418386518955
Epoch 6/100, Loss: 0.36577415838837624
Epoch 7/100, Loss: 0.3941432163119316
Epoch 8/100, Loss: 0.439587002620101
Epoch 9/100, Loss: 0.4401077330112457
Epoch 10/100, Loss: 0.3772455733269453
Epoch 11/100, Loss: 0.2954315263777971
Epoch 12/100, Loss: 0.3748079650104046
Epoch 13/100, Loss: 0.40573081001639366
Epoch 14/100, Loss: 0.3924034219235182
Epoch 15/100, Loss: 0.33317151479423046
Epoch 16/100, Loss: 0.5978201851248741
Epoch 17/100, Loss: 0.36167024821043015
Epoch 18/100, Loss: 0.3788159843534231
Epoch 19/100, Loss: 0.3119277935475111
Epoch 20/100, Loss: 0.2934840973466635
Epoch 21/100, Loss: 0.45480391569435596
Epoch 22/100, Loss: 0.45656094141304493
Epoch 23/100, Loss: 0.38080362789332867
Epoch 24/100, Loss: 0.289045125246

**Reasoning**:
Evaluate the model after training for more epochs using the validation set and calculate the accuracy, confusion matrix, and classification report to assess performance.



In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in valid_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(preds.cpu().numpy())

# Accuracy
accuracy = accuracy_score(y_true, y_pred)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# Classification Report
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

print(f"\nAccuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Calculate and print per-class sensitivity and specificity
for i, cls in enumerate(class_names):
    TP = cm[i, i]
    FN = np.sum(cm[i, :]) - TP
    FP = np.sum(cm[:, i]) - TP
    TN = np.sum(cm) - TP - FN - FP

    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0.0

    print(f"{cls} - Sensitivity (Recall): {sensitivity:.4f}, Specificity: {specificity:.4f}")


Accuracy: 0.8571

Classification Report:
              precision    recall  f1-score   support

        Mild       0.67      0.50      0.57         4
      Normal       0.95      1.00      0.98        20
    moderate       0.50      0.67      0.57         3
      severe       0.00      0.00      0.00         1

    accuracy                           0.86        28
   macro avg       0.53      0.54      0.53        28
weighted avg       0.83      0.86      0.84        28

Mild - Sensitivity (Recall): 0.5000, Specificity: 0.9583
Normal - Sensitivity (Recall): 1.0000, Specificity: 0.8750
moderate - Sensitivity (Recall): 0.6667, Specificity: 0.9200
severe - Sensitivity (Recall): 0.0000, Specificity: 1.0000


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


## Summary:

### Data Analysis Key Findings
*   After training for 100 epochs with the initial model architecture, the overall accuracy was 0.6786.
*   Applying data augmentation techniques to the training data was successfully implemented.
*   Experimenting with different learning rates on the initial model architecture showed that a learning rate of 0.001 yielded the highest validation accuracy of 0.7500 after 20 epochs.
*   Testing different optimizers on the initial model architecture revealed that the RMSprop optimizer achieved the highest accuracy of 0.7500 after 20 epochs with the best learning rate.
*   Adjusting the model architecture by using a pre-trained ResNet-50 model with frozen parameters was successfully implemented.
*   Fine-tuning the learning rate for the ResNet-50 model showed that both 0.001 and 0.0001 learning rates achieved an accuracy of 0.8214 after 20 epochs, with 0.001 being identified as the best.
*   Experimenting with different optimizers for the ResNet-50 model indicated that the Adam optimizer achieved the highest accuracy of 0.8571 after 20 epochs with the best learning rate.
*   Evaluating the ResNet-50 model trained with the best configuration (Adam optimizer, learning rate 0.001) for 100 epochs resulted in a validation accuracy of 0.8571.
*   The classification report for the final model showed an overall accuracy of 0.86. Per-class metrics varied significantly, with the 'Normal' class having high recall (1.00) and the 'severe' class having a sensitivity (recall) of 0.0000.

### Insights or Next Steps
*   The model's performance on the 'severe' class is a major weakness, significantly impacting overall metrics like the macro average F1-score (0.56). Future efforts should focus on improving the model's ability to classify this class, potentially through techniques like oversampling the minority class, using focal loss, or exploring different model architectures more suited for imbalanced datasets.
*   Despite implementing various techniques like increasing epochs, data augmentation, and using a pre-trained model with hyperparameter tuning, the target accuracy of 89.3% was not met. Further investigation into more advanced techniques such as transfer learning with unfreezing some layers of the pre-trained model, exploring different pre-trained architectures, or implementing more sophisticated data augmentation strategies could be beneficial.
